In [1]:
!pip install transformers torch librosa soundfile jiwer -q

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Aug 19 19:28:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
#!pip uninstall torch torchvision torchaudio -y

In [4]:
#!pip install torch torchvision torchaudio

In [5]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
import glob
import os
import jiwer

In [6]:
DATASET_PATH = '/content/dataset'

In [7]:
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-portuguese"

In [8]:
TARGET_SAMPLE_RATE = 16000

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando o dispositivo: {device.upper()}")

Usando o dispositivo: CUDA


In [10]:
tokenizer = Wav2Vec2Processor.from_pretrained(MODEL_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID, use_safetensors=True).to(device)

In [12]:
datapath = os.path.join(DATASET_PATH, '*.m4a')
setlist_m4a = glob.glob(datapath)

In [13]:
referencias = {
    'a1.m4a': 'EU ESTOU AVALIANDO MODELOS DE APRENDIZADO DE VOZ PARA UM EXPERIMENTO CIENTÍFICO',
    'a2.m4a': 'O DIA ESTÁ ENSOLARADO MAS ACHO QUE DAQUI A POUCO IRÁ CHOVER',
    'a3.m4a': 'BRASÍLIA É UMA CIDADE PLANEJADA',
    'a4.m4a': 'A CAPITAL DA TURQUIA NÃO É ISTAMBUL',
    'a5.m4a': 'TODAS AS NOITES ANTES DE DORMIR EU COLOCO MEU PIJAMA',
    'a6.m4a': 'TODAS AS MANHÃS EU CORRO PARA NÃO PERDER O ÔNIBUS',
    'a7.m4a': 'MEU CACHORRO É MUITO BRINCALHÃO',
    'a8.m4a': 'A OBRA PRIMA DE CAMÕES É OS LUSÍADAS',
    'a9.m4a': 'OS LUSÍADAS SÃO A OBRA PRIMA DE CAMÕES',
    'a10.m4a': 'TESTANDO TESTANDO UM DOS TRÊS TESTANDO',
    'a11.m4a': 'DENTRE AS CIVILIZAÇÕES DA ANTIGUIDADE ESTÃO OS GREGOS OS EGÍPCIOS OS SUMÉRIOS OS ASSÍRIOS E OS FENÍCIOS'
}

In [14]:
all_predictions = []
all_references = []

In [15]:
if not setlist_m4a:
    print(f"\nERRO: Nenhum arquivo .m4a encontrado em '{DATASET_PATH}'")
else:
    print(f"\n{len(setlist_m4a)} arquivo(s) .m4a encontrados. Começando a transcrição e avaliação...")
    print("-" * 40)

    for caminho_do_m4a in sorted(setlist_m4a):
        nome_do_arquivo = os.path.basename(caminho_do_m4a)
        print(f"Processando arquivo: {nome_do_arquivo}...")

        referencia_texto = referencias.get(nome_do_arquivo)
        if not referencia_texto or referencia_texto == 'TEXTO CORRETO AQUI':
            print(f"  -> AVISO: Nenhuma transcrição de referência encontrada ou preenchida para este arquivo. Pulando.")
            print("-" * 40)
            continue

        try:
            caminho_do_wav = os.path.splitext(caminho_do_m4a)[0] + '.wav'
            comando_ffmpeg = f"ffmpeg -i '{caminho_do_m4a}' -ar {TARGET_SAMPLE_RATE} -ac 1 -y '{caminho_do_wav}' -loglevel quiet"
            os.system(comando_ffmpeg)

            audio_array, sample_rate = librosa.load(caminho_do_wav, sr=None)

            input_values = tokenizer(audio_array, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
            with torch.no_grad():
                logits = model(input_values).logits
            predicted_ids = torch.argmax(logits, dim=-1)
            previsao_texto = tokenizer.batch_decode(predicted_ids)[0]

            error = jiwer.wer(referencia_texto.lower(), previsao_texto.lower())

            all_references.append(referencia_texto)
            all_predictions.append(previsao_texto)

            print(f"  -> Referência: {referencia_texto}")
            print(f"  -> Previsão  : {previsao_texto}")
            print(f"  -> WER (arquivo): {error * 100:.2f}%")

            os.remove(caminho_do_wav)
            print("-" * 40)

        except Exception as e:
            print(f"  -> Erro ao processar o arquivo {nome_do_arquivo}: {e}")
            print("-" * 40)



11 arquivo(s) .m4a encontrados. Começando a transcrição e avaliação...
----------------------------------------
Processando arquivo: a1.m4a...
  -> Referência: EU ESTOU AVALIANDO MODELOS DE APRENDIZADO DE VOZ PARA UM EXPERIMENTO CIENTÍFICO
  -> Previsão  : eu estou avaliando modelos de aprendizado de voz para um experimento científico
  -> WER (arquivo): 0.00%
----------------------------------------
Processando arquivo: a10.m4a...
  -> Referência: TESTANDO TESTANDO UM DOS TRÊS TESTANDO
  -> Previsão  : testando testando um dos três testando
  -> WER (arquivo): 0.00%
----------------------------------------
Processando arquivo: a11.m4a...
  -> Referência: DENTRE AS CIVILIZAÇÕES DA ANTIGUIDADE ESTÃO OS GREGOS OS EGÍPCIOS OS SUMÉRIOS OS ASSÍRIOS E OS FENÍCIOS
  -> Previsão  : dentre as civilizações danticheguidades tão os gregos os egípcios os sumérios os assírios e os fenícios
  -> WER (arquivo): 17.65%
----------------------------------------
Processando arquivo: a2.m4a...
  -> Referê

### Dataset

In [16]:
!pip install datasets torchcodec

In [17]:
#!apt-get update && apt-get install -y ffmpeg

In [18]:
#!pip install --upgrade datasets torch torchaudio

In [19]:
from datasets import load_dataset

dataset_id = "facebook/multilingual_librispeech"
config_name = "portuguese"
num_samples = 100

print(f"Tentando carregar as primeiras {num_samples} amostras do dataset '{dataset_id}', configuração '{config_name}'...")

try:
    dataset_sample = load_dataset(
        dataset_id,
        config_name,
        split=f'train[:{num_samples}]'
    )

    print("\n✅ Amostra de 100 áudios carregada com sucesso!")
    print("\nInformações do dataset carregado:")
    print(dataset_sample)

    print("\n--- Exemplo da primeira amostra (índice 0) ---")
    primeiro_exemplo = next(iter(dataset_sample))
    print(primeiro_exemplo)

    audio_data = primeiro_exemplo['audio']

    transcricao = primeiro_exemplo['transcript']

    audio_array = audio_data['array']
    sampling_rate = audio_data['sampling_rate']

    print(f"\nTranscrição: '{transcricao}'")
    print(f"Taxa de amostragem (Sample Rate): {sampling_rate} Hz")
    print(f"Formato do array de áudio (Shape): {audio_array.shape}")

    print("\n--- Iterando sobre as 3 primeiras amostras ---")
    for i, exemplo in enumerate(dataset_sample):
        if i >= 3:
            break
        print(f"Amostra {i}: {exemplo['transcript']}")

except Exception as e:
    print(f"\n❌ ERRO ao carregar ou processar o dataset: {e}")
    print("\nVerifique se o ID do dataset e o nome da configuração estão corretos.")
    print("Se o erro for de 'KeyError', verifique se os nomes das colunas ('text', 'transcript', etc.) estão corretos.")

Tentando carregar as primeiras 100 amostras do dataset 'facebook/multilingual_librispeech', configuração 'portuguese'...


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]


✅ Amostra de 100 áudios carregada com sucesso!

Informações do dataset carregado:
Dataset({
    features: ['audio', 'original_path', 'begin_time', 'end_time', 'transcript', 'audio_duration', 'speaker_id', 'chapter_id', 'file', 'id'],
    num_rows: 100
})

--- Exemplo da primeira amostra (índice 0) ---
{'audio': <datasets.features._torchcodec.AudioDecoder object at 0x7e2ef05bcb30>, 'original_path': 'http://www.archive.org/download/canaa_1901_librivox/canaa_12_aranha_64kb.mp3', 'begin_time': 617.09, 'end_time': 635.04, 'transcript': 'e quando ia sendo arrebatado pela expansão dos seus mais íntimos anceios maciel conteve-se com esforço ficou repentinamente mudo fitando com os olhos vermelhos e húmidos o extrangeiro', 'audio_duration': 17.949999999999932, 'speaker_id': '12249', 'chapter_id': '10229', 'file': '12249_10229_000000.opus', 'id': '12249_10229_000000'}

Transcrição: 'e quando ia sendo arrebatado pela expansão dos seus mais íntimos anceios maciel conteve-se com esforço ficou repe

In [20]:
#!pip uninstall -y torch torchvision torchaudio torchcodec

In [21]:
#!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

In [22]:
#!pip install torchcodec

In [23]:
from IPython.display import Audio

audio_array = primeiro_exemplo['audio']['array']
sampling_rate = primeiro_exemplo['audio']['sampling_rate']

print("\nOuvindo o primeiro áudio:")
Audio(audio_array, rate=sampling_rate)


Ouvindo o primeiro áudio:


In [24]:
import torch
import jiwer
from datasets import load_dataset

try:
    dataset = load_dataset('facebook/multilingual_librispeech', 'portuguese', split='test', streaming=True)
    dataset_head = dataset.take(100)
    print("✅ Amostras do dataset carregadas com sucesso!")

except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    exit()


all_references = []
all_predictions = []

print(f"\nComeçando a transcrição e avaliação de 100 amostras...")
print("-" * 50)

for i, sample in enumerate(dataset_head):
    try:
        audio_input = sample['audio']['array']
        sample_rate = sample['audio']['sampling_rate']
        referencia_texto = sample['transcript']

        TARGET_SAMPLE_RATE = 16000
        if sample_rate != TARGET_SAMPLE_RATE:
            pass

        input_values = tokenizer(audio_input, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt").input_values.to(device)

        with torch.no_grad():
            logits = model(input_values).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        previsao_texto = tokenizer.batch_decode(predicted_ids)[0]

        referencia_lower = referencia_texto.lower()
        previsao_lower = previsao_texto.lower()
        all_references.append(referencia_lower)
        all_predictions.append(previsao_lower)

        error = jiwer.wer(referencia_lower, previsao_lower)

        print(f"--- Amostra {i+1} ---")
        print(f" -> Referência: {referencia_lower}")
        print(f" -> Previsão  : {previsao_lower}")
        print(f" -> WER (amostra): {error * 100:.2f}%")
        print("-" * 50)

    except Exception as e:
        print(f" -> Erro ao processar a amostra {i+1}: {e}")
        print("-" * 50)

# --- CÁLCULO FINAL DA WER ---
if all_references and all_predictions:
    wer_geral = jiwer.wer(all_references, all_predictions)
    print("\n" + "="*50)
    print("AVALIAÇÃO FINALIZADA")
    print(f"Total de amostras processadas: {len(all_references)}")
    print(f"WER Geral (em todo o conjunto de teste): {wer_geral * 100:.2f}%")
    print("="*50)
else:
    print("\nNenhuma amostra foi processada. A avaliação não pôde ser concluída.")

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

✅ Amostras do dataset carregadas com sucesso!

Começando a transcrição e avaliação de 100 amostras...
--------------------------------------------------
--- Amostra 1 ---
 -> Referência: foice cortou o pescoço do animal a cabeça ficou segura na carne da vítima e das artérias rotas jorrava o sangue não havia mais cães a matar o terreiro ficara alastrado de corpos decepados mutilados de membros esparsos os homens maltratados doloridos deitavam
 -> Previsão  : lau u pashushlarnãoataasofa pruxa meraara na naa chuha aa shop a axo mutahruvo ono chaga abaxh a ha pano a a roa ala plohaga auasashaponomuito choloámaboa jashplojul mas rui pca pagrum no aaaata
 -> WER (amostra): 95.35%
--------------------------------------------------
--- Amostra 2 ---
 -> Referência: névoa fugiu o céu espanou-se e dilatou-se em maravilhosa limpidez a mancha móvel sobre a planície definiu-se no perfil de um pobre cavalo que passeava na verdura os seus olhos de velhice e fadiga tristes e longos de passada com os
 

### Fine-Tuning

In [25]:
from datasets import load_dataset, Audio

train_dataset = load_dataset("facebook/multilingual_librispeech", "portuguese", split="train")
test_dataset = load_dataset("facebook/multilingual_librispeech", "portuguese", split="test")

train_dataset = train_dataset.select(range(400))
test_dataset = test_dataset.select(range(100))
# -----------------------------------------

train_dataset = train_dataset.remove_columns(['speaker_id', 'chapter_id', 'original_path', 'begin_time', 'end_time', 'audio_duration', 'file'])
test_dataset = test_dataset.remove_columns(['speaker_id', 'chapter_id', 'original_path', 'begin_time', 'end_time', 'audio_duration', 'file'])

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

In [26]:
import json
from datasets import ClassLabel
import re

# Função para extrair todos os caracteres únicos do texto
def extract_all_chars(batch):
    all_text = " ".join(batch["transcript"])
    # Regex para remover caracteres especiais, exceto letras, apóstrofo e espaço
    vocab = list(set(re.sub(r'[^\w\s\']', '', all_text).lower()))
    return {"vocab": [vocab], "all_text": [all_text]}

# Gera o vocabulário
vocabs = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)

# Cria a lista final de caracteres e adiciona tokens especiais
vocab_list = list(set(vocabs["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

# Adiciona o caractere de "desconhecido" e o de "padding"
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
# O CTC usa o token 0 como "blank", então vamos garantir que o padding não seja o 0
if vocab_dict.get('[PAD]') == 0:
    # Troca o token de padding com outro para evitar o conflito
    pad_token_id = vocab_dict['[PAD]']
    other_token = list(vocab_dict.keys())[1]
    other_token_id = vocab_dict[other_token]
    vocab_dict[other_token] = pad_token_id
    vocab_dict['[PAD]'] = other_token_id

print(f"Vocabulário criado com {len(vocab_dict)} tokens: {sorted(vocab_dict.keys())}")


# Salva o vocabulário em um arquivo json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Cria o Processor, que combina o tokenizer e o feature extractor
from transformers import Wav2Vec2Processor, Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(
    "./vocab.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    word_delimiter_token=" ",
)

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=False
)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Vocabulário criado com 41 tokens: [' ', "'", '[PAD]', '[UNK]', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', 'á', 'â', 'ã', 'ç', 'é', 'ê', 'í', 'ò', 'ó', 'ô', 'õ', 'ú']


In [27]:
# O modelo Wav2Vec2 espera áudio em 16kHz
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    audio = batch["audio"]
    # Processa o áudio
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

    # Processa o texto
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcript"].lower()).input_ids

    return batch

# Aplica a função de pré-processamento em todo o dataset
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [30]:
from huggingface_hub import notebook_login

notebook_login()

In [37]:
import numpy as np
import torch
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import jiwer

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

# 2. Métrica de Avaliação (WER)
wer_metric = jiwer

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.wer(label_str, pred_str)

    return {"wer": wer}


model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

model.freeze_feature_encoder()

training_args = TrainingArguments(
  output_dir="wav2vec2-finetuned-portuguese",
  group_by_length=True,
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  do_eval=True,
  num_train_epochs=10,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=50,
  eval_steps=50,
  logging_steps=50,
  learning_rate=5e-5,
  weight_decay=0.005,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3425196517.py:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [44]:
training_args = TrainingArguments(
  output_dir="wav2vec2-finetuned-portuguese-final",
  group_by_length=True,

  per_device_train_batch_size=1,      # << O MENOR VALOR POSSÍVEL
  gradient_accumulation_steps=8,      # << (1 * 8 = 8). Mantém o batch size efetivo de 8
  # ------------------------------------

  per_device_eval_batch_size=2,
  num_train_epochs=15,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-5,
  weight_decay=0.005,
  warmup_steps=100,
  save_total_limit=2,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,
)

trainer.train()

/tmp/ipython-input-1696220866.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss
100,2.869400
200,2.859100
300,2.800300
400,2.668300
500,2.539300
600,2.386300
700,2.254600


/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:188: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

TrainOutput(global_step=750, training_loss=2.5970687154134113, metrics={'train_runtime': 3931.7258, 'train_samples_per_second': 1.526, 'train_steps_per_second': 0.191, 'total_flos': 2.761051915957968e+18, 'train_loss': 2.5970687154134113, 'epoch': 15.0})

In [45]:
output_dir = "wav2vec2-finetuned-portuguese-final"

trainer.save_model(output_dir)
processor.save_pretrained(output_dir)

print(f"✅ Modelo e processador salvos com sucesso em: '{output_dir}'")

✅ Modelo e processador salvos com sucesso em: 'wav2vec2-finetuned-portuguese-final'


In [46]:
import torch
import jiwer
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

MODEL_PATH = "wav2vec2-finetuned-portuguese-final"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Carregando o processador de: {MODEL_PATH}...")
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)

print(f"Carregando o modelo de: {MODEL_PATH}...")
model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH).to(device)

# -DATASET DE TESTE ---
dataset = load_dataset(
    'facebook/multilingual_librispeech', 'portuguese',
    split='test', streaming=True
)
dataset_head = dataset.take(100)
print("✅ Dataset de teste carregado!")

# ---  AVALIAÇÃO ---
all_references = []
all_predictions = []

print("\nIniciando a avaliação final...")
for sample in dataset_head:
    audio_input = sample['audio']['array']
    referencia_texto = sample['transcript'].lower()

    input_values = processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values.to(device)

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    previsao_texto = processor.batch_decode(predicted_ids)[0].lower()

    all_references.append(referencia_texto)
    all_predictions.append(previsao_texto)

# --- CÁLCULO FINAL DA WER ---
wer_geral = jiwer.wer(all_references, all_predictions)

print("\n" + "="*50)
print("AVALIAÇÃO FINALIZADA")
print(f"✅ WER Geral (modelo final): {wer_geral * 100:.2f}%")
print("="*50)

# Mostra alguns exemplos para ver a qualidade
print("\n--- Exemplos de Transcrições ---")
for i in range(min(5, len(all_references))):
    print(f"Referência: {all_references[i]}")
    print(f"Previsão  : {all_predictions[i]}\n")

Carregando o processador de: wav2vec2-finetuned-portuguese-final...
Carregando o modelo de: wav2vec2-finetuned-portuguese-final...


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

✅ Dataset de teste carregado!

Iniciando a avaliação final...

AVALIAÇÃO FINALIZADA
✅ WER Geral (modelo final): 100.00%

--- Exemplos de Transcrições ---
Referência: foice cortou o pescoço do animal a cabeça ficou segura na carne da vítima e das artérias rotas jorrava o sangue não havia mais cães a matar o terreiro ficara alastrado de corpos decepados mutilados de membros esparsos os homens maltratados doloridos deitavam
Previsão  : aoaoooaaaaaaoaoaaaaaoaaaaaoaooaaaaaaaaaoaaaadoaoaaaaaoooaadooaoaado aaomoaaa

Referência: névoa fugiu o céu espanou-se e dilatou-se em maravilhosa limpidez a mancha móvel sobre a planície definiu-se no perfil de um pobre cavalo que passeava na verdura os seus olhos de velhice e fadiga tristes e longos de passada com os
Previsão  : aoaaraaaooaaaaaaaaaaoodaaooaoaaaaaaaaaaaaoodaraooaoaaoaaoooaaoaaaaaaaaaaaaoao

Referência: viviam unidos em uma só comunhão de desânimo e de espanto na casinha feita de madeira tosca com teto de telhas de pau incendiada pelo sol n

In [39]:
processor.save_pretrained("asl-wav2vec2-modelo-fine-tuned")

[]

In [41]:


print("Realizando um teste rápido com o modelo em memória...")

model_em_memoria = trainer.model
processor_em_memoria = processor # O mesmo processador usado no treino

# Pega uma única amostra do seu dataset de teste
amostra_teste = test_dataset.select([10])[0] # 11ª amostra

# Extrai o áudio e a referência
audio_input = amostra_teste["input_values"]
referencia_texto = processor_em_memoria.decode(amostra_teste["labels"])

# Converte o áudio para um tensor PyTorch e move para a GPU/CPU
input_tensor = torch.tensor(audio_input).unsqueeze(0).to(device)

# Faz a previsão com o modelo em memória
with torch.no_grad():
    logits = model_em_memoria(input_tensor).logits

predicted_ids = torch.argmax(logits, dim=-1)
previsao_texto = processor_em_memoria.batch_decode(predicted_ids)[0]

print("\n" + "="*50)
print("RESULTADO DO TESTE RÁPIDO")
print(f"\n -> Texto de Referência: '{referencia_texto.lower()}'")
print(f" -> Texto Previsto     : '{previsao_texto.lower()}'")
print("="*50)

Realizando um teste rápido com o modelo em memória...


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



RESULTADO DO TESTE RÁPIDO

 -> Texto de Referência: 'admirou ao longe o corte das montanhas a negrura da mata a fronte das árvores debaixo dos seus pés a tera vermelha como embebida de sangue e das plantas tenebrosas o cheiro que tonteia e excita'
 -> Texto Previsto     : ''


In [40]:
import torch
import jiwer
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


MODEL_PATH = "asl-wav2vec2-modelo-fine-tuned"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Carregando o processador do diretório: {MODEL_PATH}...")
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)

print(f"Carregando o seu modelo fine-tuned: {MODEL_PATH}...")
model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH).to(device)

try:
    dataset = load_dataset(
        'facebook/multilingual_librispeech',
        'portuguese',
        split='test',
        streaming=True
    )
    dataset_head = dataset.take(100) # Avaliando nas mesmas 100 amostras
    print("✅ Amostras do dataset de teste carregadas com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    exit()

# AVALIAÇÃO DO MODELO ---
all_references = []
all_predictions = []

print(f"\nComeçando a avaliação de 100 amostras com o seu modelo treinado...")
print("-" * 50)

for i, sample in enumerate(dataset_head):
    audio_input = sample['audio']['array']
    referencia_texto = sample['transcript'].lower()

    input_values = processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values.to(device)

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    previsao_texto = processor.batch_decode(predicted_ids)[0].lower()

    all_references.append(referencia_texto)
    all_predictions.append(previsao_texto)


    if (i + 1) % 10 == 0:
        print(f"Processada amostra {i+1}...")

# --- CÁLCULO FINAL DA WER ---
wer_geral = jiwer.wer(all_references, all_predictions)

print("\n" + "="*50)
print("AVALIAÇÃO FINALIZADA")
print(f"Total de amostras processadas: {len(all_references)}")
print(f"✅ WER Geral (com modelo fine-tuned): {wer_geral * 100:.2f}%")
print("="*50)

# alguns exemplos para ver a qualidade
print("\n--- Exemplos de Transcrições ---")
for i in range(5):
    print(f"Referência: {all_references[i]}")
    print(f"Previsão  : {all_predictions[i]}\n")

Carregando o processador do diretório: asl-wav2vec2-modelo-fine-tuned...
Carregando o seu modelo fine-tuned: asl-wav2vec2-modelo-fine-tuned...


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

✅ Amostras do dataset de teste carregadas com sucesso!

Começando a avaliação de 100 amostras com o seu modelo treinado...
--------------------------------------------------
Processada amostra 10...
Processada amostra 20...
Processada amostra 30...
Processada amostra 40...
Processada amostra 50...
Processada amostra 60...
Processada amostra 70...
Processada amostra 80...
Processada amostra 90...
Processada amostra 100...

AVALIAÇÃO FINALIZADA
Total de amostras processadas: 100
✅ WER Geral (com modelo fine-tuned): 100.00%

--- Exemplos de Transcrições ---
Referência: foice cortou o pescoço do animal a cabeça ficou segura na carne da vítima e das artérias rotas jorrava o sangue não havia mais cães a matar o terreiro ficara alastrado de corpos decepados mutilados de membros esparsos os homens maltratados doloridos deitavam
Previsão  : 

Referência: névoa fugiu o céu espanou-se e dilatou-se em maravilhosa limpidez a mancha móvel sobre a planície definiu-se no perfil de um pobre cavalo que 

In [33]:
trainer.save_model("asl-wav2vec2-modelo-fine-tuned")
processor.save_pretrained("asl-wav2vec2-modelo-fine-tuned")

# Envia o resultado final para o Hub
trainer.push_to_hub()

# Como usar seu novo modelo para inferência
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="asl-wav2vec2-modelo-fine-tuned")

# Transcreva um novo arquivo de áudio (exemplo)
# audio_file = "caminho/para/seu/audio.wav"
# pipe(audio_file)

Device set to use cuda:0


In [36]:
processor.save_pretrained("asl-wav2vec2-modelo-fine-tuned")

print("✅ Arquivos do processador salvos/corrigidos com sucesso em 'asl-wav2vec2-modelo-fine-tuned'.")

✅ Arquivos do processador salvos/corrigidos com sucesso em 'asl-wav2vec2-modelo-fine-tuned'.


In [35]:
import torch
import jiwer
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


MODEL_PATH = "./wav2vec2-finetuned-portuguese/checkpoint-1000"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Carregando o processador do diretório: {MODEL_PATH}...")
# O processador (com o vocabulário correto) também é salvo no checkpoint
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)

print(f"Carregando o seu modelo fine-tuned: {MODEL_PATH}...")
model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH).to(device)

# --- 2. CARREGAMENTO DO MESMO DATASET DE TESTE ---
try:
    dataset = load_dataset(
        'facebook/multilingual_librispeech',
        'portuguese',
        split='test',
        streaming=True
    )
    dataset_head = dataset.take(100)
    print("✅ Amostras do dataset de teste carregadas com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    exit()

# --- 3. AVALIAÇÃO DO MODELO (LÓGICA IDÊNTICA) ---
all_references = []
all_predictions = []

print(f"\nComeçando a transcrição e avaliação de 100 amostras com o seu modelo...")
print("-" * 50)

for i, sample in enumerate(dataset_head):
    try:
        audio_input = sample['audio']['array']
        referencia_texto = sample['transcript']

        input_values = processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values.to(device)

        with torch.no_grad():
            logits = model(input_values).logits

        predicted_ids = torch.argmax(logits, dim=-1)

        pad_token_id = processor.tokenizer.pad_token_id
        print(f" -> ID do token de preenchimento (pad): {pad_token_id}")
        print(f" -> IDs previstos (raw): {predicted_ids[0].tolist()}")

        previsao_texto = processor.batch_decode(predicted_ids)[0]

        referencia_lower = referencia_texto.lower()
        previsao_lower = previsao_texto.lower()

        all_references.append(referencia_lower)
        all_predictions.append(previsao_lower)

        error = jiwer.wer(referencia_lower, previsao_lower)
        print(f"--- Amostra {i+1} ---")
        print(f" -> Referência: {referencia_lower}")
        print(f" -> Previsão  : {previsao_lower}")
        print(f" -> WER (amostra): {error * 100:.2f}%")
        print("-" * 50)

    except Exception as e:
        print(f" -> Erro ao processar a amostra {i+1}: {e}")
        print("-" * 50)

# --- CÁLCULO FINAL DA WER ---
if all_references and all_predictions:
    wer_geral = jiwer.wer(all_references, all_predictions)
    print("\n" + "="*50)
    print("AVALIAÇÃO FINALIZADA")
    print(f"Total de amostras processadas: {len(all_references)}")
    print(f"WER Geral (com modelo fine-tuned): {wer_geral * 100:.2f}%")
    print("="*50)
else:
    print("\nNenhuma amostra foi processada.")

Carregando o processador do diretório: ./wav2vec2-finetuned-portuguese/checkpoint-1000...


TypeError: expected str, bytes or os.PathLike object, not NoneType